<a href="https://colab.research.google.com/github/ethanmarshallanalytics/Deep-Learning-and-Neural-Networks/blob/main/MATH_6373_Homework_5_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

from google.colab import drive


drive.mount('/content/drive/')
%ls
%cd drive
%ls
%cd MyDrive/
%ls



Mounted at /content/drive/
drive/  sample_data/
/content/drive
MyDrive/
/content/drive/MyDrive
unzip:  cannot find or open /content/drive/MyDrive/cat_v1.zip, /content/drive/MyDrive/cat_v1.zip.zip or /content/drive/MyDrive/cat_v1.zip.ZIP.


In [6]:
%ls

'2020 Qualified Athletes.gsheet'   IMG_4126.MOV
 Bracketology.gsheet               irisData.csv
 cat_v1/                          'Jersey Boys Restaurant.gdoc'
'cat_v1 (2)'/                     'NBA Team Roster.gsheet'
'Classic Rock .gsheet'            'NCAA College Basketball'/
'Colab Notebooks'/                'Red Sox 2019 Depthhart.gsheet'
'Copy of IMG_1749.MOV'            'Summer Training Program.gsheet'
'Course Schedule.gsheet'           sWOlmDz8RE.mp4
'Courses Spring 2020.gsheet'      'Untitled document.gdoc'
 CS167/                           'Untitled spreadsheet.gsheet'
 dTE2Q1buUf.mp4                   'USA Outdoor Championship Qualifiers.gsheet'
'Getting started.pdf'              vxCAsrg4Bg.mp4
 image.jpg                         ZipShare/


In [9]:
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

dataset = torchvision.datasets.ImageFolder(root='/content/drive/MyDrive/cats-breed-dataset/', transform=preprocess)
dataset



Dataset ImageFolder
    Number of datapoints: 298
    Root location: /content/drive/MyDrive/cats-breed-dataset/
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=True)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained AlexNet model
model = torchvision.models.alexnet(pretrained=True)

# Freeze the parameters of the feature extraction layers
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer for 5 classes
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 5)

# Move the model to GPU if available
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Load Cats breed dataset and split into train and test sets
# Assuming you have already downloaded and organized the dataset
# You need to define the dataset loading part accordingly

# Assuming train_loader and test_loader are PyTorch DataLoader objects

# Define the number of epochs
num_epochs = 10

# Lists to store training and testing loss, and testing accuracy
train_loss_history = []
test_loss_history = []
test_accuracy_history = []

# Training the model
for epoch in range(num_epochs):
    # Set model to training mode
    model.train()

    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Compute average training loss for the epoch
    train_loss = running_loss / len(train_loader)
    train_loss_history.append(train_loss)

    # Set model to evaluation mode
    model.eval()

    # Evaluate on test set
    test_running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Compute average testing loss and accuracy for the epoch
    test_loss = test_running_loss / len(test_loader)
    test_loss_history.append(test_loss)
    accuracy = correct / total * 100
    test_accuracy_history.append(accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, "
          f"Test Loss: {test_loss:.4f}, "
          f"Test Accuracy: {accuracy:.2f}%")

# Plotting training and test loss
plt.plot(range(1, num_epochs+1), train_loss_history, label='Training Loss')
plt.plot(range(1, num_epochs+1), test_loss_history, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.show()

# Plotting test accuracy
plt.plot(range(1, num_epochs+1), test_accuracy_history, label='Test Accuracy', color='green')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Test Accuracy')
plt.legend()
plt.show()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 80.2MB/s]


NameError: name 'train_loader' is not defined